In [9]:
import pandas as pd
import numpy as np

import pickle
from dateutil.parser import parse
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
from time import time
from sklearn.metrics import accuracy_score

In [2]:
PROCESSED_DATA_FOLDER = "data/4_all_data_preprocessed"
TRAIN_DATASET_FILE = "train_dataset.pkl"
TEST_DATASET_FILE = "test_dataset.pkl"

RESULT_DIR = "data/hyperparams_tune"

In [3]:
with open(f"{PROCESSED_DATA_FOLDER}/{TRAIN_DATASET_FILE}", 'rb') as handle:
    df_train = pickle.load(handle)
    
with open(f"{PROCESSED_DATA_FOLDER}/{TEST_DATASET_FILE}", 'rb') as handle:
    df_test = pickle.load(handle)


In [4]:
# LOAD TRAIN AND TEST DATA
# PLEASE UPDATE BEFORE USING REST OF THE FILE
# IN CASE YOU HAVE TE DATASETS STORED ELSEWHERE

df_train = df_train.fillna(0)
train_y = df_train.event_indicator
y = df_train.event_indicator.astype(float)
train_x = df_train.drop(["isw_date_tomorrow_datetime", "day_datetime", "city_resolvedAddress", "event_indicator"], axis=1)

df_test = df_test.fillna(0)
test_y = df_test.event_indicator
y = df_test.event_indicator.astype(float)
test_x = df_test.drop(["isw_date_tomorrow_datetime", "day_datetime", "city_resolvedAddress", "event_indicator"], axis=1)

In [5]:
def tune_with_halving_grid_search(x_train, y_train, param_grid, model, result_name):
    start = time()
    halving_gs_results = HalvingGridSearchCV(
        model,
        param_grid,
        cv=3,
        factor=3,
        min_resources='exhaust',
        verbose=10
    ).fit(x_train, y_train)

    duration = time() - start

    results = pd.DataFrame(halving_gs_results.cv_results_)
    results.loc[:, 'mean_test_score'] *= 100

    # take the most relevant columns and sort
    results = results.loc[:, ('iter', 'rank_test_score', 'mean_test_score', 'params')]
    results.sort_values(by=['iter', 'rank_test_score'], ascending=[False, True], inplace=True)
    results.to_csv(f"{RESULT_DIR}/{result_name}.csv")
    
    return results, duration

In [6]:
def perform_tune(param_grid, model, result_name):
    #halving_results, halving_duration = tune_with_halving_grid_search(train_x[10000:].head(500), train_y[10000:].head(500), param_grid, model, result_name)
    halving_results, halving_duration = tune_with_halving_grid_search(train_x[100:], train_y[100:], param_grid, model, result_name)
    print(halving_results.head())

    score = halving_results['mean_test_score'].iloc[0]
    params = halving_results['params'].iloc[0]

    print(f'Best score for HalvingGridSearchCv is {score:.3f}, took {halving_duration:.2f} seconds')
    print(f'Params: {params}')

In [15]:
# Logistic regression

model = LogisticRegression(max_iter=1000)
solvers = ['newton-cg', 'lbfgs', 'liblinear']
penalty = ['l2']
c_values = [100, 10, 1.0, 0.1, 0.01]

param_grid = dict(solver=solvers,penalty=penalty,C=c_values)

perform_tune(param_grid, model, 'LogisticRegression_Hyperparams')

n_iterations: 3
n_required_iterations: 3
n_possible_iterations: 3
min_resources_: 18095
max_resources_: 162860
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 15
n_resources: 18095
Fitting 3 folds for each of 15 candidates, totalling 45 fits
[CV 1/3; 1/15] START C=100, penalty=l2, solver=newton-cg........................


C:\Python310\lib\site-packages\sklearn\utils\validation.py:768: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
C:\Python310\lib\site-packages\scipy\optimize\_linesearch.py:457: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Python310\lib\site-packages\scipy\optimize\_linesearch.py:306: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Python310\lib\site-packages\sklearn\utils\optimize.py:203: UserWarning: Line Search failed
  warnings.warn("Line Search failed")
C:\Python310\lib\site-packages\sklearn\utils\validation.py:768: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
C:\Python310\lib\site-packages\sklearn\utils\validation.py:768: UserWarning: pandas.DataFrame with spars

[CV 1/3; 1/15] END C=100, penalty=l2, solver=newton-cg;, score=(train=0.871, test=0.419) total time= 2.8min
[CV 2/3; 1/15] START C=100, penalty=l2, solver=newton-cg........................


C:\Python310\lib\site-packages\sklearn\utils\validation.py:768: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
C:\Python310\lib\site-packages\scipy\optimize\_linesearch.py:457: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Python310\lib\site-packages\scipy\optimize\_linesearch.py:306: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Python310\lib\site-packages\scipy\optimize\_linesearch.py:416: LineSearchWarning: Rounding errors prevent the line search from converging
  warn(msg, LineSearchWarning)
C:\Python310\lib\site-packages\sklearn\utils\optimize.py:203: UserWarning: Line Search failed
  warnings.warn("Line Search failed")
C:\Python310\lib\site-packages\sklearn\utils\validation.py:768: UserWarning: pandas.DataFrame with sparse columns 

[CV 2/3; 1/15] END C=100, penalty=l2, solver=newton-cg;, score=(train=0.880, test=0.553) total time= 2.4min
[CV 3/3; 1/15] START C=100, penalty=l2, solver=newton-cg........................


...

[CV 2/3; 2/2] END C=0.1, penalty=l2, solver=liblinear;, score=(train=0.854, test=0.770) total time=  34.2s
[CV 3/3; 2/2] START C=0.1, penalty=l2, solver=liblinear.........................


C:\Python310\lib\site-packages\sklearn\utils\validation.py:768: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
C:\Python310\lib\site-packages\sklearn\utils\validation.py:768: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
C:\Python310\lib\site-packages\sklearn\utils\validation.py:768: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


[CV 3/3; 2/2] END C=0.1, penalty=l2, solver=liblinear;, score=(train=0.842, test=0.757) total time=  29.0s


C:\Python310\lib\site-packages\sklearn\utils\validation.py:768: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


    iter  rank_test_score  mean_test_score   
20     2                5        76.547706  \
21     2                8        76.286117   
19     1               13        75.665006   
16     1               14        75.583964   
17     1               15        75.554477   

                                               params  
20  {'C': 1.0, 'penalty': 'l2', 'solver': 'libline...  
21  {'C': 0.1, 'penalty': 'l2', 'solver': 'libline...  
19  {'C': 0.1, 'penalty': 'l2', 'solver': 'libline...  
16  {'C': 1.0, 'penalty': 'l2', 'solver': 'libline...  
17  {'C': 0.01, 'penalty': 'l2', 'solver': 'liblin...  
Best score for HalvingGridSearchCv is 76.548, took 2220.16 seconds
Params: {'C': 1.0, 'penalty': 'l2', 'solver': 'liblinear'}


In [ ]:
res = estimator.fit(train_x.tail(100), train_y.tail(100))
print(res)
accuracy = accuracy_score(test_y, res.predict(test_x))
accuracy

In [16]:
# Support Vector Machines

model = SVC()
kernel = ['poly', 'rbf', 'sigmoid']
C = [50, 10, 1.0, 0.1, 0.01]
gamma = ['scale']
tol = [1e-3, 1e-2]

param_grid = dict(kernel=kernel,C=C,gamma=gamma,tol=tol)

perform_tune(param_grid, model, 'SupportVectorMachine_Hyperparams')

n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 6031
max_resources_: 162860
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 30
n_resources: 6031
Fitting 3 folds for each of 30 candidates, totalling 90 fits
[CV 1/3; 1/30] START C=50, gamma=scale, kernel=poly, tol=0.001..................


C:\Python310\lib\site-packages\sklearn\utils\validation.py:768: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
C:\Python310\lib\site-packages\sklearn\utils\validation.py:768: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
C:\Python310\lib\site-packages\sklearn\utils\validation.py:768: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


[CV 1/3; 1/30] END C=50, gamma=scale, kernel=poly, tol=0.001;, score=(train=0.734, test=0.716) total time=  10.2s
[CV 2/3; 1/30] START C=50, gamma=scale, kernel=poly, tol=0.001..................


C:\Python310\lib\site-packages\sklearn\utils\validation.py:768: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
C:\Python310\lib\site-packages\sklearn\utils\validation.py:768: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
C:\Python310\lib\site-packages\sklearn\utils\validation.py:768: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


[CV 2/3; 1/30] END C=50, gamma=scale, kernel=poly, tol=0.001;, score=(train=0.731, test=0.714) total time=   9.0s
[CV 3/3; 1/30] START C=50, gamma=scale, kernel=poly, tol=0.001..................


...

C:\Python310\lib\site-packages\sklearn\utils\validation.py:768: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
C:\Python310\lib\site-packages\sklearn\utils\validation.py:768: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
C:\Python310\lib\site-packages\sklearn\utils\validation.py:768: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


[CV 1/3; 3/4] END C=1.0, gamma=scale, kernel=rbf, tol=0.01;, score=(train=0.728, test=0.729) total time= 9.9min
[CV 2/3; 3/4] START C=1.0, gamma=scale, kernel=rbf, tol=0.01....................


C:\Python310\lib\site-packages\sklearn\utils\validation.py:768: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
C:\Python310\lib\site-packages\sklearn\utils\validation.py:768: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
C:\Python310\lib\site-packages\sklearn\utils\validation.py:768: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


[CV 2/3; 3/4] END C=1.0, gamma=scale, kernel=rbf, tol=0.01;, score=(train=0.726, test=0.727) total time=10.5min
[CV 3/3; 3/4] START C=1.0, gamma=scale, kernel=rbf, tol=0.01....................


KeyboardInterrupt: 

In [15]:
# Stochastic Gradient Descent
model = SGDClassifier()
penalty = [None, 'l2', 'l1', 'elasticnet']
alpha = [1e-4, 1e-3, 1e-2]
tol = [1e-3, 1e-2]

param_grid = dict(penalty=penalty,alpha=alpha,tol=tol)

perform_tune(param_grid, model, 'StochasticGradientDescent_Hyperparams')

n_iterations: 3
n_required_iterations: 3
n_possible_iterations: 3
min_resources_: 18095
max_resources_: 162860
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 24
n_resources: 18095
Fitting 3 folds for each of 24 candidates, totalling 72 fits
[CV 1/3; 1/24] START alpha=0.0001, penalty=None, tol=0.001......................


C:\Python310\lib\site-packages\sklearn\utils\validation.py:768: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
C:\Python310\lib\site-packages\sklearn\utils\validation.py:768: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
C:\Python310\lib\site-packages\sklearn\utils\validation.py:768: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


[CV 1/3; 1/24] END alpha=0.0001, penalty=None, tol=0.001;, score=(train=0.277, test=0.288) total time=   4.9s
[CV 2/3; 1/24] START alpha=0.0001, penalty=None, tol=0.001......................


C:\Python310\lib\site-packages\sklearn\utils\validation.py:768: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
C:\Python310\lib\site-packages\sklearn\utils\validation.py:768: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
C:\Python310\lib\site-packages\sklearn\utils\validation.py:768: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


[CV 2/3; 1/24] END alpha=0.0001, penalty=None, tol=0.001;, score=(train=0.728, test=0.731) total time=   6.6s
[CV 3/3; 1/24] START alpha=0.0001, penalty=None, tol=0.001......................


...

[CV 2/3; 3/3] END alpha=0.0001, penalty=l1, tol=0.01;, score=(train=0.715, test=0.732) total time= 2.7min
[CV 3/3; 3/3] START alpha=0.0001, penalty=l1, tol=0.01..........................


C:\Python310\lib\site-packages\sklearn\utils\validation.py:768: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
C:\Python310\lib\site-packages\sklearn\utils\validation.py:768: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
C:\Python310\lib\site-packages\sklearn\utils\validation.py:768: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


[CV 3/3; 3/3] END alpha=0.0001, penalty=l1, tol=0.01;, score=(train=0.790, test=0.730) total time= 5.9min


C:\Python310\lib\site-packages\sklearn\utils\validation.py:768: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


    iter  rank_test_score  mean_test_score   
34     2                1        74.995996  \
32     2               12        69.698032   
33     2               29        55.570086   
28     1                3        73.028854   
30     1                4        72.625449   

                                             params  
34  {'alpha': 0.0001, 'penalty': 'l1', 'tol': 0.01}  
32  {'alpha': 0.0001, 'penalty': None, 'tol': 0.01}  
33  {'alpha': 0.0001, 'penalty': 'l2', 'tol': 0.01}  
28  {'alpha': 0.0001, 'penalty': 'l1', 'tol': 0.01}  
30  {'alpha': 0.0001, 'penalty': 'l2', 'tol': 0.01}  
Best score for HalvingGridSearchCv is 74.996, took 3402.22 seconds
Params: {'alpha': 0.0001, 'penalty': 'l1', 'tol': 0.01}


In [14]:
# Random Forest
model = RandomForestClassifier()
n_estimators = [10, 100, 1000]
max_features = ['sqrt', 'log2', 'auto']
max_depth = [10, 50, 100, None]
min_samples_leaf = [1, 2, 4]

param_grid = dict(n_estimators=n_estimators,max_features=max_features,max_depth=max_depth,min_samples_leaf=min_samples_leaf)

perform_tune(param_grid, model, 'RandomForest_Hyperparams')

n_iterations: 5
n_required_iterations: 5
n_possible_iterations: 5
min_resources_: 2010
max_resources_: 162860
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 108
n_resources: 2010
Fitting 3 folds for each of 108 candidates, totalling 324 fits
[CV 1/3; 1/108] START max_depth=10, max_features=sqrt, min_samples_leaf=1, n_estimators=10


C:\Python310\lib\site-packages\sklearn\utils\validation.py:768: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
C:\Python310\lib\site-packages\sklearn\utils\validation.py:768: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
C:\Python310\lib\site-packages\sklearn\utils\validation.py:768: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


[CV 1/3; 1/108] END max_depth=10, max_features=sqrt, min_samples_leaf=1, n_estimators=10;, score=(train=0.928, test=0.321) total time=   0.5s
[CV 2/3; 1/108] START max_depth=10, max_features=sqrt, min_samples_leaf=1, n_estimators=10


C:\Python310\lib\site-packages\sklearn\utils\validation.py:768: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
C:\Python310\lib\site-packages\sklearn\utils\validation.py:768: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
C:\Python310\lib\site-packages\sklearn\utils\validation.py:768: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


[CV 2/3; 1/108] END max_depth=10, max_features=sqrt, min_samples_leaf=1, n_estimators=10;, score=(train=0.963, test=0.563) total time=   0.6s
[CV 3/3; 1/108] START max_depth=10, max_features=sqrt, min_samples_leaf=1, n_estimators=10


...

[CV 2/3; 2/2] END max_depth=100, max_features=auto, min_samples_leaf=1, n_estimators=10;, score=(train=0.998, test=0.525) total time=  20.5s
[CV 3/3; 2/2] START max_depth=100, max_features=auto, min_samples_leaf=1, n_estimators=10


C:\Python310\lib\site-packages\sklearn\utils\validation.py:768: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
C:\Python310\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Python310\lib\site-packages\sklearn\utils\validation.py:768: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
C:\Python310\lib\site-packages\sklearn\utils\validation.py:768: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


[CV 3/3; 2/2] END max_depth=100, max_features=auto, min_samples_leaf=1, n_estimators=10;, score=(train=0.998, test=0.731) total time=  19.6s


C:\Python310\lib\site-packages\sklearn\utils\validation.py:768: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
C:\Python310\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


     iter  rank_test_score  mean_test_score   
161     4              100        54.044427  \
160     4              162        50.897813   
159     3               79        54.698227   
157     3              136        53.237017   
156     3              141        53.098831   

                                                params  
161  {'max_depth': 100, 'max_features': 'auto', 'mi...  
160  {'max_depth': None, 'max_features': 'auto', 'm...  
159  {'max_depth': 100, 'max_features': 'auto', 'mi...  
157  {'max_depth': None, 'max_features': 'auto', 'm...  
156  {'max_depth': 50, 'max_features': 'sqrt', 'min...  
Best score for HalvingGridSearchCv is 54.044, took 1888.40 seconds
Params: {'max_depth': 100, 'max_features': 'auto', 'min_samples_leaf': 1, 'n_estimators': 10}
